In [1]:
#@title Initialise
from manim import *
import numpy as np

_vec = [[1, 0, 0], [0, 1, 0], [0, 0, 1]]
_rand = [[1, 1, 2], [1, 2, 4], [-1, 0, 1]]
_vec2 = np.dot(_vec, np.transpose(_rand))
_rect = [[1, 1, 2],[1, 2, 4]]

_symm = [[2, 1, 1], [1, 2, 1], [1, 1, 4]]

def update(m):
    global w, v, wD, wS, wDS, vS, vT, vTS
    w, v = np.linalg.eig(m)
    wD = np.diag(w);
    def shorten(e):
        e3 = round(e, 2)
        e = round(e)
        if (e == e3):
            return e
        else:
            return e3
    wS = list(map(shorten, w))
    wDS = np.diag(wS);
    vS = list(map(lambda e: list(map(shorten, e)), v))
    vT = np.transpose(v)
    vTS = list(map(lambda e: list(map(shorten, e)), vT))
    
def vec2arr(v, colors=[None, None, None]):
    return list(map(lambda ie:
            Arrow3D(
                color=colors[ie[0]],
                start=np.array([0, 0, 0]),
                end=np.array(ie[1]),
                resolution=8)
            , enumerate(v)))

Manim Community v0.9.0

# Spectral Decomposition and Single Value Decomposition

by Rowan Clarke, Aditi Dhillon, Alessandro Curioni, Ethan Darbyshire, Jonathan Baccari

## Transformations

We can visualise matrices by thier transformations on the identity.

In [2]:
#@title Visualise Matrix as a Transformation

mat = _rand
vec = _vec
vec2 = _vec2

class VectorTransformation(ThreeDScene):
    def construct(self):
        ax = ThreeDAxes()
        colors = [RED, GREEN, BLUE]
        arrs_before = vec2arr(vec)
        arrs_after = vec2arr(vec2, colors)
        m = Matrix(mat).to_corner(UL)
        self.add_fixed_in_frame_mobjects(m)
        self.set_camera_orientation(phi=2*PI/5, theta=-PI/5)
        self.add(ax, *arrs_before)
        self.begin_3dillusion_camera_rotation()
        self.play(Create(m))
        self.play(m.animate.set_column_colors(*colors), *list(map(lambda arri: arri[1].animate.set_color(colors[arri[0]]), enumerate(arrs_before))))
        self.wait()
        self.play(*list(map(lambda arri: arri[1].animate.become(arrs_after[arri[0]]), enumerate(arrs_before))))
        self.wait(duration=3)
        
%manim -qm -o vmt -v WARNING VectorTransformation

In [3]:
#@title Transforming a Sphere

mat = _rand

class SphereTransformation(ThreeDScene):
    def construct(self):
        ax = ThreeDAxes()
        m = Matrix(mat).to_corner(UL)
        self.add_fixed_in_frame_mobjects(m)
        self.set_camera_orientation(phi=2*PI/5, theta=PI/5)
        sphere = Sphere()
        self.add(ax, sphere)
        self.begin_3dillusion_camera_rotation()
        self.play(ApplyMatrix(mat, sphere))
        self.wait(duration=3)
        
%manim -qm -o st -v WARNING SphereTransformation

In [4]:
#@title Diagonal Matrices are Enlargements 
mat = [[5, 0, 0], [0, 3, 0], [0, 0, 1]]

%manim -qm -o ste -v WARNING SphereTransformation

In [5]:
#@title Orthogonal Matrices are Rotations 
mat = [[1, 0, 0], [0, 0, -1], [0, 1, 0]]

%manim -qm -o sto -v WARNING SphereTransformation

## Spectral Decomposition

> Q: Why are orthogonal and diagonal matrices interesting? (Hint: Spectral Decomposition)

Because for all real symmetric matrices $A$, there exists an orthoganal matrix $P$ and diagonal matrix $D$ such that $$A=PDP^\top$$

So all real symmetric matrices can be represented by a rotation, enlargement, and rotation (where the last rotation is the inverse of the first.)

In [6]:
#@title Visualise Matrix as a Transformation

mat = _symm

%manim -qm -o svmt -v WARNING SphereTransformation

In [7]:
#@title Spectral Decomposition

mat = _symm

labs = ["A", "D", "P", r"P^\top"]

class SpectralDecomposition(Scene):
    def construct(self):
        update(mat)
        def mat2str(matrix):
            return r"\left[\begin{matrix}" + r"\\".join(map(lambda r: "&".join(map(lambda e: str(e), r)), matrix)) + r"\end{matrix}\right]"

        m = MathTex(labs[0] + r"=" + mat2str(mat)).move_to([-4.5,2.5,0])
        t = MathTex(r"c_A(x)=" + "".join(map(lambda e: "(" + str(e) + " - " + "x)", wS))).next_to(m, buff=1)
        d = MathTex(labs[1] + r"=" + mat2str(wDS)).move_to([-4,0.5,0])
        p = MathTex(labs[2] + r"=" + mat2str(vS)).next_to(d, buff=1)
        pT = MathTex(labs[3] + r"=" + mat2str(vTS)).next_to(p, direction=DOWN, buff=0.5)
        
        self.add(m)
        self.wait()
        self.play(Create(t))
        self.play(Transform(t.copy(), d), Create(p))
        self.play(Transform(p.copy(), pT))
        self.wait(duration=3)

%manim -qm -o sd -v WARNING SpectralDecomposition

In [8]:
#@title Transformation of Eigenvectors

mat = _symm
update(mat)
vec = vT
vec2 = np.dot(vec, np.transpose(_symm))

class SphereTransformationVectors(ThreeDScene):
    def construct(self):
        ax = ThreeDAxes()
        colors = [RED, GREEN, BLUE]
        arrs_before = vec2arr(vec, colors)
        arrs_after = vec2arr(vec2, colors)
        m = Matrix(mat).to_corner(UL)
        self.add_fixed_in_frame_mobjects(m)
        self.set_camera_orientation(phi=2*PI/5, theta=PI/5)
        sphere = Sphere(fill_opacity=0.4)
        self.add(ax, sphere)
        self.begin_3dillusion_camera_rotation()
        self.play(ApplyMatrix(mat, sphere), *list(map(lambda arri: arri[1].animate.become(arrs_after[arri[0]]), enumerate(arrs_before))), run_time=2)
        self.wait(duration=3)

%manim -qm -o sde -v WARNING SphereTransformationVectors

Nice!

As we can see, the transformation of a matrix is just an enlargement along its eigenvectors.
So we can decompose it into:
 - Change the basis of the eigenvectors to the identity.
 - Enlarge by the eigenvalues.
 - Change the basis back to the eigenvectors.
 
And this is exactly what we see for real-symmetric matrices, where the change of basis is just a rotation.

In [9]:
#@title Transformations of Spectral Decomposition

mat = _symm

class SphereSpectralDecomposition(ThreeDScene):
    def construct(self):
        update(mat)
        ax = ThreeDAxes()
        p = Matrix(vS).to_corner(UL)
        d = Matrix(wDS).to_corner(UL)
        pT = Matrix(vTS).to_corner(UL)
        self.set_camera_orientation(phi=2*PI/5, theta=-PI/5)
        sphere = Sphere()
        self.add(ax, sphere)
        self.begin_3dillusion_camera_rotation()
        self.add_fixed_in_frame_mobjects(pT)
        self.play(FadeIn(pT), ApplyMatrix(vT, sphere))
        self.wait()
        self.add_fixed_in_frame_mobjects(d)
        self.play(FadeOut(pT), FadeIn(d), ApplyMatrix(wD, sphere))
        self.wait()
        self.add_fixed_in_frame_mobjects(p)
        self.play(FadeOut(d), FadeIn(p), ApplyMatrix(v, sphere))
        self.wait(duration=3)
        
%manim -qm -o sdt -v WARNING SphereSpectralDecomposition

## Single Value Decomposition

So that's cool! However...

> Pick any matrix. It's likely to be non-symmetric.

... it only works on real symmetric matrices.

If only there was a way to decompose any real matrix into a rotation, enlargement, and rotation.

There is!

We need to find an orthogonal matrix $V$ such that $AV$ has orthogonal vectors, or rather $$AV=U\Sigma$$ where $\Sigma$ is diagonal and $U$ is orthogonal.

Then we have $$A=U\Sigma V^\top$$

## A Return to Spectral Decomposition

Consider $AA^\top$.

$AA^\top = (U\Sigma V^\top)(U\Sigma V^\top)^\top=(U\Sigma V^\top)(V\Sigma^\top U^\top)$

So $AA^\top = U\Sigma\Sigma^\top U^\top$ because $V^{-1}=V^\top$

> Notice what happened to the dimension.

Since $AA^\top$ is symmetric, we can use spectral decomposition!

$U$ is the orthonormal basis of eigenvectors and $\Sigma\Sigma^\top$ is the diagonal matrix of eigenvalues of $AA^\top$. 

We can do the same for $A^\top A$ and find that $V$ is the orthonormal basis of eigenvectors and $\Sigma^\top\Sigma$ is the diagonal matrix of eigenvalues of $A^\top A$.

> So what is $\Sigma$? We know it's diagonal, right?

But $\Sigma$ is rectangular. Since the non-zero eigenvalues of $A^\top A$ and $A A^\top$ are the same, $\Sigma$ is the **rectangular** diagonal matrix of the roots of the non-zero eigenvalues.


In [10]:
mat = _rect

class ShowAATranspose(Scene):
    def construct(self):
        def mat2str(matrix):
            return r"\left[\begin{matrix}" + r"\\".join(map(lambda r: "&".join(map(lambda e: str(e), r)), matrix)) + r"\end{matrix}\right]"

        m = MathTex(r"A=" + mat2str(mat)).move_to([0,2.5,0])
        mmt = MathTex(r"AA^\top=" + mat2str(np.dot(mat, np.transpose(mat)))).next_to(m, direction=DOWN+LEFT)
        mtm = MathTex(r"A^\top A=" + mat2str(np.dot(np.transpose(mat), mat))).next_to(m, direction=DOWN+RIGHT)
        
        self.add(m)
        self.wait()
        self.play(Transform(m.copy(), mmt))
        self.play(Transform(m.copy(), mtm))
        self.wait(duration=3)
        
%manim -qm -o aat -v WARNING ShowAATranspose

In [11]:
#@title Spectral Decomposition for U

mat = np.dot(_rect, np.transpose(_rect))

labs = [r"AA^\top", r"\Sigma\Sigma^\top", "U", r"U^\top"]

%manim -qm -o aatu -v WARNING SpectralDecomposition

In [12]:
#@title Spectral Decomposition for V

mat = np.dot(np.transpose(_rect), _rect)

labs = [r"A^\top A", r"\Sigma^\top\Sigma", "V", r"V^\top"]

%manim -qm -o aatv -v WARNING SpectralDecomposition

> Notice how the non-zero eigenvalues are the same?

## But why?

If $A$ represents a massive dataset, then the orthogonal basis vectors in $U$ corresponding to the largest eigenvalues is the weighting of each dimension which have the most profound effect on the data. This process is called Principle Component Analysis. 